In [1]:
import os
import io
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import datetime
from pandas.tseries.offsets import BMonthEnd

#dask
import dask.dataframe as dd

import warnings
warnings.filterwarnings("ignore")

### Household

In [2]:
# eciw semi info 
path = r"\\VNCRP1002\ud03$\X266754\My Documents\Lapse\Data"
# read all the files with extension .csv
filenames = glob.glob(path + "\*household*.csv")
print(filenames)

['\\\\VNCRP1002\\ud03$\\X266754\\My Documents\\Lapse\\Data\\vul_eciw_household_monthly_2022_1.csv', '\\\\VNCRP1002\\ud03$\\X266754\\My Documents\\Lapse\\Data\\vul_eciw_household_monthly_2022_2.csv', '\\\\VNCRP1002\\ud03$\\X266754\\My Documents\\Lapse\\Data\\vul_eciw_household_monthly_2022_3.csv', '\\\\VNCRP1002\\ud03$\\X266754\\My Documents\\Lapse\\Data\\vul_eciw_household_monthly_2022_4.csv', '\\\\VNCRP1002\\ud03$\\X266754\\My Documents\\Lapse\\Data\\vul_eciw_household_monthly_2022_5.csv', '\\\\VNCRP1002\\ud03$\\X266754\\My Documents\\Lapse\\Data\\vul_eciw_household_monthly_2022_6.csv']


In [3]:
#list of all df
dfs = []
for f in filenames: 
    print(f)
    #d = pd.read_csv(f, encoding="ISO-8859-1", header=None, skiprows=1, na_values=" ") 
    d = pd.read_csv(f, encoding="ISO-8859-1", na_values=" ")
    dfs.append(d) 
df_hh = pd.concat(dfs, ignore_index=True)
del dfs

\\VNCRP1002\ud03$\X266754\My Documents\Lapse\Data\vul_eciw_household_monthly_2022_1.csv
\\VNCRP1002\ud03$\X266754\My Documents\Lapse\Data\vul_eciw_household_monthly_2022_2.csv
\\VNCRP1002\ud03$\X266754\My Documents\Lapse\Data\vul_eciw_household_monthly_2022_3.csv
\\VNCRP1002\ud03$\X266754\My Documents\Lapse\Data\vul_eciw_household_monthly_2022_4.csv
\\VNCRP1002\ud03$\X266754\My Documents\Lapse\Data\vul_eciw_household_monthly_2022_5.csv
\\VNCRP1002\ud03$\X266754\My Documents\Lapse\Data\vul_eciw_household_monthly_2022_6.csv


In [4]:
# df_hh = df_hh.drop_duplicates()

In [4]:
df_hh['as_of_hist_dt']=df_hh['as_of_hist_dt'].astype('datetime64[ns]')
df_hh['hhld_acq_dt']=df_hh['hhld_acq_dt'].astype('datetime64[ns]')
df_hh['most_rcnt_pur_dt']=df_hh['most_rcnt_pur_dt'].astype('datetime64[ns]')

In [5]:
df_hh = df_hh[['household_id', 
               #dates
               'as_of_hist_dt', 'hhld_acq_dt', 'most_rcnt_pur_dt',
               # codes
               'incm_rng_cd', 'prim_occup_cd','prmr_prsnc_chld_ind', 'net_wrth_gold_cd','hhld_qdrnt_cd',
               #asset mix
              'wc_tot_asset_amt','wc_asset_mix_anty_amt','wc_asset_mix_bond_amt',
               'wc_asset_mix_dep_amt','wc_asset_mix_mf_amt','wc_asset_mix_oth_asset_amt',
               'wc_asset_mix_stk_amt']]

In [6]:
# min_half_date= df_hh['as_of_hist_dt'].min()
# max_half_date= df_hh['as_of_hist_dt'].max()

### Subject

In [6]:
# eciw monthly info 
path = r"\\VNCRP1002\ud03$\X266754\My Documents\Lapse\Data"
# read all the files with extension .csv
filenames = glob.glob(path + "\*subject*.csv")

In [7]:
# drop product id
dfs = []
for f in filenames: 
    print(f)
    #d = pd.read_csv(f, encoding="ISO-8859-1", header=None, skiprows=1, na_values=" ") 
    d= pd.read_csv(f, encoding="ISO-8859-1", na_values=" ")
    dfs.append(d) 
df_sb = pd.concat(dfs, ignore_index=True)
del dfs

\\VNCRP1002\ud03$\X266754\My Documents\Lapse\Data\vul_eciw_subject_monthly_2022_1.csv
\\VNCRP1002\ud03$\X266754\My Documents\Lapse\Data\vul_eciw_subject_monthly_2022_2.csv
\\VNCRP1002\ud03$\X266754\My Documents\Lapse\Data\vul_eciw_subject_monthly_2022_3.csv
\\VNCRP1002\ud03$\X266754\My Documents\Lapse\Data\vul_eciw_subject_monthly_2022_4.csv
\\VNCRP1002\ud03$\X266754\My Documents\Lapse\Data\vul_eciw_subject_monthly_2022_5.csv
\\VNCRP1002\ud03$\X266754\My Documents\Lapse\Data\vul_eciw_subject_monthly_2022_6.csv


In [8]:
# pd.set_option('display.max_rows', None)
# df_sb.dtypes

In [9]:
df_sb = df_sb.sort_values(['subject_id', 'as_of_hist_dt'])
df_sb = df_sb.drop_duplicates()

In [10]:
df_sb['as_of_hist_dt']=df_sb['as_of_hist_dt'].astype('datetime64[ns]')
df_sb['subj_acq_dt']=df_sb['subj_acq_dt'].astype('datetime64[ns]')

In [12]:
df_sb = df_sb[['household_id', 'subject_id', 'agmt_pkge_id','policy_id',
               'as_of_hist_dt', 'subj_acq_dt',
               'mrtl_stat_cd', 'gndr_cd', 'subj_state_cd', 'subj_act_ind',
               'pref_lang_cd','deceased_ind', 'rtl_dstrb_orphn_cd',
               'ixi_buoyancy', 'ixi_fin_cohort_cd',
               'subj_aum_amt' ]]

### Merge

In [13]:
df =df_sb.merge(df_hh, how='left', 
                left_on=['household_id', 'as_of_hist_dt'], 
                right_on = ['household_id', 'as_of_hist_dt'],
                suffixes=('', '_remove'))
df.drop([i for i in df.columns if 'remove' in i],axis=1, inplace=True)

In [14]:
# df =df.drop(['half_date', 'as_of_hist_dt_y'], axis=1)
# df.columns = df.columns.str.replace('as_of_hist_dt_x', 'as_of_hist_dt')

In [14]:
# remove dups
#del df['prod_type_cd']

df = df.drop_duplicates()

In [15]:
pd.set_option('display.max_columns', None)
df.head()

,household_id,subject_id,agmt_pkge_id,policy_id,as_of_hist_dt,subj_acq_dt,mrtl_stat_cd,gndr_cd,subj_state_cd,subj_act_ind,pref_lang_cd,deceased_ind,rtl_dstrb_orphn_cd,ixi_buoyancy,ixi_fin_cohort_cd,subj_aum_amt,hhld_acq_dt,most_rcnt_pur_dt,incm_rng_cd,prim_occup_cd,prmr_prsnc_chld_ind,net_wrth_gold_cd,hhld_qdrnt_cd,wc_tot_asset_amt,wc_asset_mix_anty_amt,wc_asset_mix_bond_amt,wc_asset_mix_dep_amt,wc_asset_mix_mf_amt,wc_asset_mix_oth_asset_amt,wc_asset_mix_stk_amt
0,23.0,1147.0,960527100000,96052710,2022-01-28,1994-12-22,1.0,F,DE,1,1,1.0,1,1.0,M46,13258.07,1969-06-19,1995-01-03,0.0,NaN,NaN,NaN,10.0,30440.0,61.0,0.0,29163.0,751.0,16.0,449.0
2,23.0,1147.0,960527100000,96052710,2022-02-25,1994-12-22,1.0,F,DE,1,1,1.0,1,1.0,M46,12875.85,1969-06-19,1995-01-03,0.0,NaN,NaN,NaN,10.0,30440.0,61.0,0.0,29163.0,751.0,16.0,449.0
4,23.0,1147.0,960527100000,96052710,2022-03-25,1994-12-22,1.0,F,DE,1,1,1.0,1,1.0,M46,13523.57,1969-06-19,1995-01-03,0.0,NaN,NaN,NaN,10.0,30440.0,61.0,0.0,29163.0,751.0,16.0,449.0
6,23.0,1147.0,960527100000,96052710,2022-04-29,1994-12-22,1.0,F,DE,1,1,1.0,1,1.0,M46,12445.60,1969-06-19,1995-01-03,0.0,NaN,NaN,NaN,10.0,30440.0,61.0,0.0,29163.0,751.0,16.0,449.0
8,23.0,1147.0,960527100000,96052710,2022-05-27,1994-12-22,1.0,F,DE,1,1,1.0,1,1.0,M46,11401.31,1969-06-19,1995-01-03,0.0,NaN,NaN,NaN,10.0,30440.0,61.0,0.0,29163.0,751.0,16.0,449.0


In [16]:
print(df.shape)

(4753268, 30)


In [26]:
# # missing %
# df_missing = pd.DataFrame(df.isnull().sum()/len(df)).reset_index().rename(columns = {"index":"Columns", 0:"Missing%"})
# df_missing.out = df_missing.to_csv("eciw_missing_percent.csv",  header=True)

#### export to parquet

In [17]:
del df_hh,  df_sb

In [18]:
df.to_parquet('eciw_jan_june.parquet')